# 解析3D Tiles中二进制文件

In [34]:
class B3DM:
    def __init__(self, bytes):
        self.bytes = bytes
        self.parse()
    def parse(self):
        self.magic = str(self.bytes[0:4])
        self.version = int.from_bytes(self.bytes[4:8], 'little')
        self.byteLength = int.from_bytes(self.bytes[8:12], 'little')
        self.ftJsonLen = int.from_bytes(self.bytes[12:16], 'little')
        self.ftBinLen = int.from_bytes(self.bytes[16:20], 'little')
        self.btJsonLen = int.from_bytes(self.bytes[20:24], 'little')
        self.btBinLen = int.from_bytes(self.bytes[24:28], 'little')
    def ftJsonData(self):
        return self.bytes[28:28+self.ftJsonLen]
    def ftBinData(self):
        return self.bytes[28+self.ftJsonLen:28+self.ftJsonLen+self.ftBinLen]
    def btJsonData(self):
        return self.bytes[28+self.ftJsonLen+self.ftBinLen:28+self.ftJsonLen+self.ftBinLen+self.btJsonLen]
    def btBinData(self):
        return self.bytes[28+self.ftJsonLen+self.ftBinLen+self.btJsonLen:28+self.ftJsonLen+self.ftBinLen+self.btJsonLen+self.btBinLen]
    def glTFData(self):
        return self.bytes[
            28+self.ftJsonLen+self.ftBinLen+self.btJsonLen+self.btBinLen:
            self.byteLength
        ]
    def saveGlTFData(self, path):
        with open(path, 'wb') as fout:
            fout.write(self.glTFData())
        

In [35]:

class CompositeTile:
    def __init__(self, bytes):
        self.bytes = bytes
        self.parse()
    def parse(self):
        self.magic = str(self.bytes[0:4])
        self.version = int.from_bytes(self.bytes[4:8], 'little')
        self.byteLength = int.from_bytes(self.bytes[8:12], 'little')
        self.tilesLen = int.from_bytes(self.bytes[12:16], 'little')
        cur = 16
        self.tiles = []
        for i in range(self.tilesLen):
            the_magic = self.bytes[cur:cur+4]
            the_len = int.from_bytes(self.bytes[cur+8:cur+12], 'little')
            tile = None
            if the_magic==b'cmpt':
                self.tiles.append(CompositeTile(self.bytes[cur:cur+the_len]))
            elif the_magic==b'b3dm':
                self.tiles.append(B3DM(self.bytes[cur:cur+the_len]))
            elif the_magic==b'i3dm':
                # TODO
                pass
            elif the_magic==b'pnts':
                # TODO
                pass
            else:
                print("ERROR")
            cur += the_len   

In [36]:
psfsPath = "../data/tiles3d/PSFS/0composite0.cmpt"
with open(psfsPath,'rb') as f:
    bytes = f.read()
magic = bytes[0:4]
tile = None
if magic==b'cmpt':
    tile = CompositeTile(bytes)
elif magic==b'b3dm':
    tile = B3DM(bytes) 
elif magic==b'i3dm':
    # TODO
    pass
elif magic==b'pnts':
    # TODO
    pass
else:
    print("ERROR")
tile

b'cmpt'


In [37]:
tile.tiles[0].saveGlTFData('../data/gltf/psfs/tile1.glb')

In [38]:
tile.tiles[1].saveGlTFData('../data/gltf/psfs/tile2.glb')